# 🧠 Atlas — Interactive Demo

Welcome! This notebook walks you through the Atlas long-form memory system step by step.

**What you'll learn:**
1. How the agent extracts memories from conversation
2. How memories are stored and retrieved
3. How to test memory recall
4. How to use different providers (Ollama, Gemini, Groq)

> 📊 For full benchmarks and evaluation results, see **[results.ipynb](results.ipynb)**

---

## 1. Setup

First, install dependencies and load the environment.

In [1]:
import os
import sys
import json
from pathlib import Path

# Add project root to path
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.agent import LongMemAgent
from dotenv import load_dotenv

load_dotenv()

print('✓ Dependencies loaded')
print(f'  Project root: {project_root}')

/home/pik/dev/atlas/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Dependencies loaded
  Project root: /home/pik/dev/atlas


## 2. Choose Your Provider

Atlas supports multiple LLM backends. Pick one:

| Provider | Model | API Key Needed | Notes |
|----------|-------|:--------------:|-------|
| `ollama` | `mistral` | No | Local, fast, free. Run `ollama serve` first |
| `gemini` | `gemma-3-27b-it` | `GEMINI_API_KEY` | Free tier, 30 req/min |
| `groq` | `llama-3.1-8b-instant` | `GROQ_API_KEY` | Very fast, rate limited |

Edit the cell below to change providers.

In [2]:
# ═══════════════════════════════════════════
# 🔧 CONFIGURE YOUR PROVIDER HERE
# ═══════════════════════════════════════════

PROVIDER = 'groq'                 # 'ollama', 'gemini', or 'groq'
# MODEL    = 'mistral'                # model name for your provider
# MODEL  = 'gemma-3-27b-it'         # uncomment for Gemini
MODEL  = 'moonshotai/kimi-k2-instruct-0905'   # uncomment for Groq

# ═══════════════════════════════════════════

# Clean up any previous demo database
demo_db = 'demo_interactive.db'
if os.path.exists(demo_db):
    os.remove(demo_db)
    print(f'🗑️  Cleaned previous {demo_db}')

# Initialize agent
agent = LongMemAgent(
    provider=PROVIDER,
    model=MODEL,
    db_path=demo_db,
    context_limit=4096,
    flush_threshold=0.70
)

print(f'\n✓ Agent initialized')
print(f'  Provider: {agent.provider}')
print(f'  Model:    {agent.model}')
print(f'  DB:       {demo_db}')

🗑️  Cleaned previous demo_interactive.db

✓ Agent initialized
  Provider: groq
  Model:    moonshotai/kimi-k2-instruct-0905
  DB:       demo_interactive.db


## 3. Chat Helper

A nice helper to display conversations with memory metadata.

In [3]:
from IPython.display import display, Markdown, HTML

def chat(message: str, show_memories=True):
    """Send a message and display the response with memory info."""
    print(f'\n You: {message}')
    
    response = agent.chat(message)
    
    print(f'Atlas: {response["response"]}')
    print(f'\n   ┌─ Turn {response["turn_id"]} │ '
          f'Context: {response["context_utilization"]} │ '
          f'Memories: {response["total_memories"]} │ '
          f'Flush: {"yes" if response["flush_triggered"] else "no"}')
    
    if show_memories and response['active_memories']:
        print(f'   │ 🧠 Retrieved {len(response["active_memories"])} memories:')
        for mem in response['active_memories']:
            print(f'   │   • {mem["content"]} '
                  f'(from turn {mem["origin_turn"]}, conf: {mem["confidence"]:.2f})')
    print('────────────────────────────────────────────────')
    
    
    return response

print('✓ Chat helper ready. Use chat("your message") below!')

✓ Chat helper ready. Use chat("your message") below!


## 4. Plant Some Memories

Let's tell the agent some facts about ourselves. These will be extracted as memories.

In [4]:
chat('Hi! My name is Jordan and I work as a data scientist in Seattle.')
chat("I'm allergic to peanuts and I follow a vegetarian diet.")
chat('My favorite programming language is Python. I also use R for statistics.')
chat('I have a daughter named Maya, she is 5 years old.')


 You: Hi! My name is Jordan and I work as a data scientist in Seattle.
Atlas: Hey Jordan! Great to meet you. How's the data science scene treating you in Seattle?

   ┌─ Turn 1 │ Context: 3% │ Memories: 0 │ Flush: no
────────────────────────────────────────────────

 You: I'm allergic to peanuts and I follow a vegetarian diet.
Atlas: Got it—peanuts off the table and vegetarian fare only. Need any dining tips in Seattle, or is there something else on your mind?

   ┌─ Turn 2 │ Context: 4% │ Memories: 0 │ Flush: no
────────────────────────────────────────────────

 You: My favorite programming language is Python. I also use R for statistics.
Atlas: Nice stack—Python for the heavy lifting and R when you need those crisp statistical summaries. Working on anything right now that’s making you switch back and forth between them?

   ┌─ Turn 3 │ Context: 6% │ Memories: 0 │ Flush: no
────────────────────────────────────────────────

 You: I have a daughter named Maya, she is 5 years old.
Atlas

{'response': 'That’s awesome—5 is such a fun age. Is Maya already showing any signs of future data-scientist curiosity, or is she more into dinosaurs and finger-painting these days?',
 'turn_id': 4,
 'context_utilization': '7%',
 'context_tokens': 300,
 'retrieval_ms': 0.1,
 'total_ms': 614.4,
 'flush_triggered': False,
 'total_flushes': 0,
 'active_memories': [],
 'total_memories': 0}

## 5. Trigger Memory Distillation

The agent normally distills memories when context hits 70% capacity.
We can also trigger it manually to extract memories right now.

In [5]:
print('⚡ Triggering memory distillation...\n')

result = agent.manual_distill()

print(f'✓ {result["message"]}')
print(f'  Memories added: {result["memories_added"]}')
print(f'  Total memories: {result["total_memories"]}')

⚡ Triggering memory distillation...



Loading weights: 100%|██████████| 103/103 [00:00<00:00, 483.11it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


[Store] Embedding model loaded on device: cuda:0
✓ Distillation complete. 8 memories extracted.
  Memories added: 8
  Total memories: 8


## 6. Inspect Stored Memories

Let's see what the distiller extracted.

In [6]:
memories = agent.get_all_memories()

if memories:
    print(f'📦 Stored Memories ({len(memories)} total):\n')
    for m in memories:
        print(f"  [{m['type']:>12}] {m['key']}: {m['value']}")
        print(f"               confidence: {m['confidence']:.2f}, from turn {m['source_turn']}")
else:
    print('No memories stored yet. Run distillation first.')

📦 Stored Memories (8 total):

  [        fact] user_name: Jordan
               confidence: 0.95, from turn 4
  [        fact] occupation: data scientist
               confidence: 0.95, from turn 4
  [        fact] location: Seattle
               confidence: 0.95, from turn 4
  [  constraint] allergy_peanuts: allergic to peanuts
               confidence: 0.95, from turn 4
  [  preference] dietary_preference: vegetarian
               confidence: 0.95, from turn 4
  [  preference] favorite_language: Python
               confidence: 0.95, from turn 4
  [        fact] uses_r: uses R for statistics
               confidence: 0.95, from turn 4
  [      entity] daughter_maya: daughter Maya, 5 years old
               confidence: 0.95, from turn 4


## 7. Test Memory Recall

Now the key test — does the agent remember what we told it?

In [7]:
chat("What's my name?")


 You: What's my name?
Atlas: Jordan

   ┌─ Turn 5 │ Context: 7% │ Memories: 8 │ Flush: no
   │ 🧠 Retrieved 2 memories:
   │   • user_name: Jordan (from turn 4, conf: 0.95)
   │   • daughter_maya: daughter Maya, 5 years old (from turn 4, conf: 0.95)
────────────────────────────────────────────────


{'response': 'Jordan',
 'turn_id': 5,
 'context_utilization': '7%',
 'context_tokens': 293,
 'retrieval_ms': 10.7,
 'total_ms': 1045.3,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [{'memory_id': 'mem_5afb288d',
   'content': 'user_name: Jordan',
   'origin_turn': 4,
   'last_used_turn': 5,
   'type': 'fact',
   'confidence': 0.95},
  {'memory_id': 'mem_ed2ba238',
   'content': 'daughter_maya: daughter Maya, 5 years old',
   'origin_turn': 4,
   'last_used_turn': 5,
   'type': 'entity',
   'confidence': 0.95}],
 'total_memories': 8}

In [8]:
chat('Suggest a dinner recipe for me.')


 You: Suggest a dinner recipe for me.
Atlas: Quick one-pan coconut chickpea spinach curry—ready in 20 min, totally vegetarian, no peanuts anywhere.

1. Sauté 1 diced onion + 2 cloves garlic in a splash of oil.  
2. Stir in 1 Tbsp mild curry powder, 1 tsp garam masala, ¼ tsp chili flakes (optional).  
3. Add 1 can chickpeas (rinsed), 1 can full-fat coconut milk, 1 cup veggie broth; simmer 8 min.  
4. Fold in 3 big handfuls baby spinach until wilted; finish with juice of ½ lime and salt to taste.  
5. Serve over rice or quinoa; top with cilantro and a drizzle of yogurt if Maya likes it creamy.

Mild, sweet, and bright—perfect for a five-year-old palate.

   ┌─ Turn 6 │ Context: 12% │ Memories: 8 │ Flush: no
   │ 🧠 Retrieved 2 memories:
   │   • dietary_preference: vegetarian (from turn 4, conf: 0.95)
   │   • daughter_maya: daughter Maya, 5 years old (from turn 4, conf: 0.95)
────────────────────────────────────────────────


{'response': 'Quick one-pan coconut chickpea spinach curry—ready in 20 min, totally vegetarian, no peanuts anywhere.\n\n1. Sauté 1 diced onion + 2 cloves garlic in a splash of oil.  \n2. Stir in 1 Tbsp mild curry powder, 1 tsp garam masala, ¼ tsp chili flakes (optional).  \n3. Add 1 can chickpeas (rinsed), 1 can full-fat coconut milk, 1 cup veggie broth; simmer 8 min.  \n4. Fold in 3 big handfuls baby spinach until wilted; finish with juice of ½ lime and salt to taste.  \n5. Serve over rice or quinoa; top with cilantro and a drizzle of yogurt if Maya likes it creamy.\n\nMild, sweet, and bright—perfect for a five-year-old palate.',
 'turn_id': 6,
 'context_utilization': '12%',
 'context_tokens': 486,
 'retrieval_ms': 24.6,
 'total_ms': 1207.8,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [{'memory_id': 'mem_97d4e5a3',
   'content': 'dietary_preference: vegetarian',
   'origin_turn': 4,
   'last_used_turn': 6,
   'type': 'preference',
   'confidence': 0.95},
  {'me

In [9]:
chat('What should I get my kid for her birthday?')


 You: What should I get my kid for her birthday?
Atlas: A build-your-own fairy-garden kit hits the sweet spot for a five-year-old: tiny wooden toadstools, felt flowers, a battery tea-light "firefly" jar, and a little watering can she can use for real plants. It’s hands-on, sparks imagination, and gives you two a weekend project together.

   ┌─ Turn 7 │ Context: 14% │ Memories: 8 │ Flush: no
   │ 🧠 Retrieved 2 memories:
   │   • daughter_maya: daughter Maya, 5 years old (from turn 4, conf: 0.95)
   │   • favorite_language: Python (from turn 4, conf: 0.95)
────────────────────────────────────────────────


{'response': 'A build-your-own fairy-garden kit hits the sweet spot for a five-year-old: tiny wooden toadstools, felt flowers, a battery tea-light "firefly" jar, and a little watering can she can use for real plants. It’s hands-on, sparks imagination, and gives you two a weekend project together.',
 'turn_id': 7,
 'context_utilization': '14%',
 'context_tokens': 572,
 'retrieval_ms': 13.6,
 'total_ms': 1007.6,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [{'memory_id': 'mem_ed2ba238',
   'content': 'daughter_maya: daughter Maya, 5 years old',
   'origin_turn': 4,
   'last_used_turn': 7,
   'type': 'entity',
   'confidence': 0.95},
  {'memory_id': 'mem_3f5d864a',
   'content': 'favorite_language: Python',
   'origin_turn': 4,
   'last_used_turn': 7,
   'type': 'preference',
   'confidence': 0.95}],
 'total_memories': 8}

In [10]:
chat('Tell me everything you remember about me.')


 You: Tell me everything you remember about me.
Atlas: You’re Jordan, a data scientist in Seattle who codes in Python and dips into R for stats. You’re vegetarian, allergic to peanuts, and dad to 5-year-old Maya.

   ┌─ Turn 8 │ Context: 15% │ Memories: 8 │ Flush: no
   │ 🧠 Retrieved 2 memories:
   │   • user_name: Jordan (from turn 4, conf: 0.95)
   │   • daughter_maya: daughter Maya, 5 years old (from turn 4, conf: 0.95)
────────────────────────────────────────────────


{'response': 'You’re Jordan, a data scientist in Seattle who codes in Python and dips into R for stats. You’re vegetarian, allergic to peanuts, and dad to 5-year-old Maya.',
 'turn_id': 8,
 'context_utilization': '15%',
 'context_tokens': 625,
 'retrieval_ms': 13.2,
 'total_ms': 806.1,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [{'memory_id': 'mem_5afb288d',
   'content': 'user_name: Jordan',
   'origin_turn': 4,
   'last_used_turn': 8,
   'type': 'fact',
   'confidence': 0.95},
  {'memory_id': 'mem_ed2ba238',
   'content': 'daughter_maya: daughter Maya, 5 years old',
   'origin_turn': 4,
   'last_used_turn': 8,
   'type': 'entity',
   'confidence': 0.95}],
 'total_memories': 8}

## 8. Try It Yourself!

Use the cell below to have your own conversation with the agent.
Each message is processed through the full memory pipeline.

In [11]:
# Type your message here:
chat('What do I do for work?')


 You: What do I do for work?
Atlas: You’re a data scientist.

   ┌─ Turn 9 │ Context: 15% │ Memories: 8 │ Flush: no
   │ 🧠 Retrieved 2 memories:
   │   • occupation: data scientist (from turn 4, conf: 0.95)
   │   • location: Seattle (from turn 4, conf: 0.95)
────────────────────────────────────────────────


{'response': 'You’re a data scientist.',
 'turn_id': 9,
 'context_utilization': '15%',
 'context_tokens': 626,
 'retrieval_ms': 13.5,
 'total_ms': 602.5,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [{'memory_id': 'mem_2f7ec706',
   'content': 'occupation: data scientist',
   'origin_turn': 4,
   'last_used_turn': 9,
   'type': 'fact',
   'confidence': 0.95},
  {'memory_id': 'mem_9e249652',
   'content': 'location: Seattle',
   'origin_turn': 4,
   'last_used_turn': 9,
   'type': 'fact',
   'confidence': 0.95}],
 'total_memories': 8}

In [12]:
# Try another:
chat('Your message here')


 You: Your message here
Atlas: Got it—thanks for the heads-up.

   ┌─ Turn 10 │ Context: 16% │ Memories: 8 │ Flush: no
   │ 🧠 Retrieved 2 memories:
   │   • user_name: Jordan (from turn 4, conf: 0.95)
   │   • location: Seattle (from turn 4, conf: 0.95)
────────────────────────────────────────────────


{'response': 'Got it—thanks for the heads-up.',
 'turn_id': 10,
 'context_utilization': '16%',
 'context_tokens': 646,
 'retrieval_ms': 14.5,
 'total_ms': 594.7,
 'flush_triggered': False,
 'total_flushes': 1,
 'active_memories': [{'memory_id': 'mem_5afb288d',
   'content': 'user_name: Jordan',
   'origin_turn': 4,
   'last_used_turn': 10,
   'type': 'fact',
   'confidence': 0.95},
  {'memory_id': 'mem_9e249652',
   'content': 'location: Seattle',
   'origin_turn': 4,
   'last_used_turn': 10,
   'type': 'fact',
   'confidence': 0.95}],
 'total_memories': 8}

## 9. Database Peek

Under the hood, memories are stored in SQLite with vector embeddings + full-text search indexes.

In [13]:
import sqlite3

conn = sqlite3.connect(demo_db)
cursor = conn.cursor()

cursor.execute('SELECT COUNT(*) FROM memories WHERE is_active = 1')
active = cursor.fetchone()[0]

cursor.execute('SELECT COUNT(*) FROM turns')
turns = cursor.fetchone()[0]

cursor.execute('''
    SELECT type, COUNT(*) as count 
    FROM memories WHERE is_active = 1 
    GROUP BY type
''')
types = cursor.fetchall()

print(f'📋 Database: {demo_db}')
print(f'   Active memories: {active}')
print(f'   Total turns: {turns}')
print(f'   Memory types:')
for type_name, count in types:
    print(f'     {type_name}: {count}')

conn.close()

📋 Database: demo_interactive.db
   Active memories: 8
   Total turns: 10
   Memory types:
     constraint: 1
     entity: 1
     fact: 4
     preference: 2


---

## Next Steps

| What | How |
|:-----|:----|
| 📊 **See benchmarks** | Open [results.ipynb](results.ipynb) |
| 💬 **Interactive CLI** | `uv run python main.py` |
| 🧪 **Run evaluation** | `uv run python eval/evaluate.py --provider gemini` |
| 🏠 **Local eval** | `uv run python eval/evaluate.py --local --model mistral` |
| 📖 **Quick eval** | `uv run python eval/evaluate.py --quick --local` |

### Architecture

```
User Message
    │
    ▼
┌─────────────────────────┐
│    Context Manager      │ ← checks if context needs flushing
│    (token counting)     │
└────────┬────────────────┘
         │ if >70% full
         ▼
┌─────────────────────────┐
│    Memory Distiller     │ ← LLM extracts memories from conversation
│    (Groq/Gemini/Ollama) │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│    Memory Store         │ ← SQLite + sqlite-vec + FTS5
│    (persist & search)   │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│    Memory Retriever     │ ← hybrid vector + keyword search
│    (RRF fusion)         │
└────────┬────────────────┘
         │
         ▼
┌─────────────────────────┐
│    System Prompt        │ ← memories injected into context
│    + LLM Response       │
└─────────────────────────┘
```

## Cleanup (Optional)

In [14]:
# Uncomment to delete the demo database:
# os.remove(demo_db)
# print(f'✓ Removed {demo_db}')

## NOTE: To check the actual metrics, check out results.ipynb